<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Chocolate Gummy Bears](#Q2)
    - Perceptron
    - Multilayer Perceptron
4. [Keras MMP](#Q3)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** A neuron is an individual node of a layer
- **Input Layer:** The input layer is where certain information is passed to the neural network
- **Hidden Layer:** A hidden layer is where information from the previous layer is transformed in some way and passed to the next layer. It is neither the input or output layer
- **Output Layer:** The output layer is where the results of the neural network end up to be used.
- **Activation:** The mapping of input(s) to output(s) for a given node
- **Backpropagation:** How weights are updated in the neural network


## 2. Chocolate Gummy Bears <a id="Q2"></a>

Right now, you're probably thinking, "yuck, who the hell would eat that?". Great question. Your candy company wants to know too. And you thought I was kidding about the [Chocolate Gummy Bears](https://nuts.com/chocolatessweets/gummies/gummy-bears/milk-gummy-bears.html?utm_source=google&utm_medium=cpc&adpos=1o1&gclid=Cj0KCQjwrfvsBRD7ARIsAKuDvMOZrysDku3jGuWaDqf9TrV3x5JLXt1eqnVhN0KM6fMcbA1nod3h8AwaAvWwEALw_wcB). 

Let's assume that a candy company has gone out and collected information on the types of Halloween candy kids ate. Our candy company wants to predict the eating behavior of witches, warlocks, and ghosts -- aka costumed kids. They shared a sample dataset with us. Each row represents a piece of candy that a costumed child was presented with during "trick" or "treat". We know if the candy was `chocolate` (or not chocolate) or `gummy` (or not gummy). Your goal is to predict if the costumed kid `ate` the piece of candy. 

If both chocolate and gummy equal one, you've got a chocolate gummy bear on your hands!?!?!
![Chocolate Gummy Bear](https://ed910ae2d60f0d25bcb8-80550f96b5feb12604f4f720bfefb46d.ssl.cf1.rackcdn.com/3fb630c04435b7b5-2leZuM7_-zoom.jpg)

In [248]:
import pandas as pd
candy = pd.read_csv('chocolate_gummy_bears.csv')

In [251]:
y.head()

0    1
1    1
2    1
3    0
4    0
Name: ate, dtype: int64

### Perceptron

To make predictions on the `candy` dataframe. Build and train a Perceptron using numpy. Your target column is `ate` and your features: `chocolate` and `gummy`. Do not do any feature engineering. :P

Once you've trained your model, report your accuracy. Explain why you could not achieve a higher accuracy with a *simple perceptron*. It's possible to achieve ~95% accuracy on this dataset.

In [250]:
# Start your candy perceptron here

X = candy[['chocolate', 'gummy']]
y = candy['ate']

In [252]:
# import numpy as np

# class Perceptron(object):
    
#     def __init__(self, niter = 10):
#         self.niter = niter
    
#     def __sigmoid(self, x):
#         return 1 / (1 + np.exp(-x))
    
#     def __sigmoid_derivative(self, x):
#         sx = self.__sigmoid(x)
#         return sx * (1-sx)

#     def fit(self, X, y):
#         """Fit training data
#         X : Training vectors, X.shape : [#samples, #features]
#         y : Target values, y.shape : [#samples]
#         """
#         # Randomly Initialize Weights
#         X['ones'] = np.ones(X.shape[0])
#         weights = np.zeros(X.shape[1])

#         for i in range(self.niter):
#             # Weighted sum of inputs / weights
#             weighted = np.dot(X, weights)
            
#             # Activate!
#             pred = self.__sigmoid(weighted)
            
#             # Calc error
#             errors = y.values - pred.reshape(-1,1)

#             # Update the Weights
#             adjustments = errors*self.__sigmoid_derivative(pred).reshape(-1,1)
#             weights += np.dot(X.T, adjustments).reshape(3)

#         self.weights = weights 

#     def predict(self, X):
#         print(self.weights)
#         print(self.weights.shape, X.shape)
#         pred = np.dot(X,self.weights)
#         return np.where(pred >= 0.5, 1, 0)
    
# p = Perceptron(niter = 100)
# p.fit(X,y)
# p.predict(X.iloc[0])

ValueError: cannot reshape array of size 30000 into shape (3,)

In [0]:
y_pred = p.predict(X)
from sklearn.metrics import accuracy_score
print(accuracy_score(y, y_pred))
y_pred.min()


In [254]:
class Perceptron(object):
  def __init__(self, rate = 0.01, niter = 10):
    self.rate = rate
    self.niter = niter

  def fit(self, X, y):
    """Fit training data
    X : Training vectors, X.shape : [#samples, #features]
    y : Target values, y.shape : [#samples]
    """

    # weights
    self.weight = np.zeros(1 + X.shape[1])

    # Number of misclassifications
    self.errors = []  # Number of misclassifications

    for i in range(self.niter):
      err = 0
      for xi, target in zip(X, y):
        delta_w = self.rate * (target - self.predict(xi))
        self.weight[1:] += delta_w * xi
        self.weight[0] += delta_w
        err += int(delta_w != 0.0)
      self.errors.append(err)
    return self

  def net_input(self, X):
    """Calculate net input"""
    return np.dot(X, self.weight[1:]) + self.weight[0]

  def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.net_input(X) >= 0.5, 1, 0)

In [255]:
p = Perceptron(niter = 100)
p.fit(X.to_numpy(),y.to_numpy())
y_pred = p.predict(X)

In [256]:
y_pred = p.predict(X)
from sklearn.metrics import accuracy_score
print(accuracy_score(y, y_pred))
y_pred.min()


0.5


1

In [51]:
# class Perceptron(object):
#   def __init__(self, rate = 0.01, niter = 10):
#     self.rate = rate
#     self.niter = niter

#   def fit(self, X, y):
#     """Fit training data
#     X : Training vectors, X.shape : [#samples, #features]
#     y : Target values, y.shape : [#samples]
#     """

#     # weights
#     self.weight = np.zeros(1 + X.shape[1])

#     # Number of misclassifications
#     self.errors = []  # Number of misclassifications

#     for i in range(self.niter):
#       err = 0
#       for xi, target in zip(X, y):
#         delta_w = self.rate * (target - self.predict(xi))
#         self.weight[1:] += delta_w * xi
#         self.weight[0] += delta_w
#         err += int(delta_w != 0.0)
#       self.errors.append(err)
#     return self

#   def net_input(self, X):
#     """Calculate net input"""
#     return np.dot(X, self.weight[1:]) + self.weight[0]

#   def predict(self, X):
#     """Return class label after unit step"""
#     return np.where(self.net_input(X) >= 0.5, 1, 0)

0.5


In [136]:
nn = NeuralNetwork()

for i in range(1000):
    if i%100==0:
        print('Epoch: ', i)
        print('Accuracy: ', accuracy_score(y, nn.predict(X.to_numpy())))
        print('Weights1: ', nn.weights1)
        print('Weights2: ', nn.weights2)
    nn.train(X.to_numpy(),y)

pred = nn.predict(X.to_numpy())

0.5


In [275]:
X = candy[['chocolate', 'gummy']]
y = pd.Series(candy['ate'])
X.head()


,chocolate,gummy
0,0,1
1,1,0
2,0,1
3,0,0
4,1,1


In [269]:
class NeuralNetwork: 
    def __init__(self):
        # Set upArchietecture 
        self.inputs = 2
        self.hiddenNodes = 3
        self.outputNodes = 1
        
        #Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes) #3x2
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes) #3x1
    
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        #Weighted sume of inputs and hidden layer
        # print('X.shape; ', X.shape)
        # print('self.weights1.shape: ', self.weights1.shape)
        self.hidden_sum = np.dot(X, self.weights1)
        
        #Acivations of weighted sum
        # print('hidden_sum.shape: ', self.hidden_sum.shape)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # print('self.activaged_hidden.shape: ', self.activated_hidden.shape)

        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #Final activation of output
        self.activated_output = [] 
        for s in self.output_sum:
            self.activated_output.append(self.sigmoid(s[0]))
        
        return np.array(self.activated_output)
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        # print('y shape: ', y.shape)
        # print('o shape: ', o.shape)
        # print('o reshaped shape: ', o.reshape(-1,1).shape)
        self.o_error = y - o #error in output
        # print('o_error shape: ', self.o_error.shape)
        # print('sigmoidPrime(o).shape: ', self.sigmoidPrime(o).shape)
        self.o_delta = np.array(self.o_error * self.sigmoidPrime(o)) # apply derivative of sigmoid to error
        # print('o_delta shape: ', self.o_delta.shape)
        
        # print('o_delta.shape: ', self.o_delta.shape)
        # print('weights2.shape: ', self.weights2.shape)
        self.z2_error = self.o_delta.reshape(-1,1).dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        # print('activated_hidden.T.shape: ', self.activated_hidden.T.shape)
        # print('o_delta.shape: ', self.o_delta.shape)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta.reshape(-1,1)) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

    def predict(self, X):
        return np.where(self.feed_forward(X) >= 0.5, 1, 0)


In [270]:
nn = NeuralNetwork()

for i in range(1000):
    if i%100==0:
        print('Epoch: ', i)
        print('Accuracy: ', accuracy_score(y, nn.predict(X.to_numpy())))
        print('Weights1: ', nn.weights1)
        print('Weights2: ', nn.weights2)
    nn.train(X.to_numpy(),y)

pred = nn.predict(X.to_numpy())

Epoch:  0
Accuracy:  0.5
Weights1:  [[-1.27041929 -0.79843408  1.01878276]
 [-0.14289034  1.54816237 -1.37344151]]
Weights2:  [[ 0.50291634]
 [-0.62044255]
 [-1.29927435]]
Epoch:  100
Accuracy:  0.5
Weights1:  [[ 17.12126716 -27.52335176 -42.73693875]
 [ 31.83900694 -11.4762905  -31.89629627]]
Weights2:  [[127.69418203]
 [216.49169073]
 [180.58560817]]
Epoch:  200
Accuracy:  0.5
Weights1:  [[ 17.12126716 -27.52335176 -42.73693875]
 [ 31.83900694 -11.4762905  -31.89629627]]
Weights2:  [[127.69418203]
 [216.49169073]
 [180.58560817]]
Epoch:  300
Accuracy:  0.5
Weights1:  [[ 17.12126716 -27.52335176 -42.73693875]
 [ 31.83900694 -11.4762905  -31.89629627]]
Weights2:  [[127.69418203]
 [216.49169073]
 [180.58560817]]
Epoch:  400
Accuracy:  0.5
Weights1:  [[ 17.12126716 -27.52335176 -42.73693875]
 [ 31.83900694 -11.4762905  -31.89629627]]
Weights2:  [[127.69418203]
 [216.49169073]
 [180.58560817]]
Epoch:  500
Accuracy:  0.5
Weights1:  [[ 17.12126716 -27.52335176 -42.73693875]
 [ 31.83900694 -

In [271]:
# pd.DataFrame(pred).describe()

class Neural_Network(object):
    def __init__(self):        
        #Define Hyperparameters
        self.inputLayerSize = 2
        self.outputLayerSize = 1
        self.hiddenLayerSize = 3
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)  
        
        return dJdW1, dJdW2
    
    #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [276]:
# ff_pred = nn.feed_forward(X.to_numpy())
from scipy import optimize
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))   
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []
        
        params0 = self.N.getParams()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS',
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [277]:
# pd.DataFrame(ff_pred).describe()

NN = Neural_Network()
T = trainer(NN)

T.train(X.to_numpy(),y.to_numpy())

print("Predicted Output: \n" + str(NN.forward(X))) 

ValueError: shapes (10000,10000) and (1,3) not aligned: 10000 (dim 1) != 1 (dim 0)

In [171]:
y.describe()

count    10000.000000
mean         0.500000
std          0.500025
min          0.000000
25%          0.000000
50%          0.500000
75%          1.000000
max          1.000000
Name: ate, dtype: float64

In [172]:
X.to_numpy()

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [1, 0]])

P.S. Don't try candy gummy bears. They're disgusting. 

## 3. Keras MMP <a id="Q3"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV or RandomSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 3 parameters in order to get a 3 on this section.

In [176]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
97,52,1,0,108,233,1,1,147,0,0.1,2,3,3,1
116,41,1,2,130,214,0,0,168,0,2.0,1,0,2,1
237,60,1,0,140,293,0,0,170,0,1.2,1,2,3,0
254,59,1,3,160,273,0,0,125,0,0.0,2,0,2,0
202,58,1,0,150,270,0,0,111,1,0.8,2,0,3,0


In [182]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

In [192]:
type(y)

pandas.core.series.Series

In [247]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Important Hyperparameters
inputs = X.shape[1]
epochs = 20
batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(inputs,)))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model

model.fit(X, y.to_numpy(), 
          validation_split=0.33, 
          epochs=epochs, 
          batch_size=batch_size
         )
# Validation accuracy is .76

Train on 203 samples, validate on 100 samples
Epoch 1/20
203/203 [==============================] - 1s 5ms/sample - loss: 3.3779 - accuracy: 0.4828 - val_loss: 2.7596 - val_accuracy: 0.4800
Epoch 2/20
203/203 [==============================] - 0s 421us/sample - loss: 3.2841 - accuracy: 0.5468 - val_loss: 2.5999 - val_accuracy: 0.5100
Epoch 3/20
203/203 [==============================] - 0s 392us/sample - loss: 3.1178 - accuracy: 0.5813 - val_loss: 2.4202 - val_accuracy: 0.5100
Epoch 4/20
203/203 [==============================] - 0s 497us/sample - loss: 2.6462 - accuracy: 0.6158 - val_loss: 2.1055 - val_accuracy: 0.5600
Epoch 5/20
203/203 [==============================] - 0s 408us/sample - loss: 2.2972 - accuracy: 0.6158 - val_loss: 1.8060 - val_accuracy: 0.6000
Epoch 6/20
203/203 [==============================] - 0s 406us/sample - loss: 2.0424 - accuracy: 0.6256 - val_loss: 1.6487 - val_accuracy: 0.6200
Epoch 7/20
203/203 [==============================] - 0s 444us/sample - loss: 1.

In [241]:
def create_model(nodes = 50, optimizer='adam'):
    # Create Model
    model = Sequential()
    model.add(Dense(nodes, activation='relu', input_shape=(inputs,)))
    model.add(Dense(nodes/2, activation='relu'))
    model.add(Dense(1))
    # Compile Model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [244]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn = create_model, verbose=0, epochs=20)

optimizer = ['SGD', 'Adam']
nodes = [10,20,30,40]
param_grid = dict(optimizer=optimizer, nodes=nodes)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)


In [245]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7788778940836588 using {'nodes': 20, 'optimizer': 'SGD'}
Means: 0.7425742546717325, Stdev: 0.042777181169124885 with: {'nodes': 10, 'optimizer': 'SGD'}
Means: 0.5841584205627441, Stdev: 0.11404062515802182 with: {'nodes': 10, 'optimizer': 'Adam'}
Means: 0.7788778940836588, Stdev: 0.02034459460670113 with: {'nodes': 20, 'optimizer': 'SGD'}
Means: 0.6699670056502024, Stdev: 0.14056461615202767 with: {'nodes': 20, 'optimizer': 'Adam'}
Means: 0.7458746035893759, Stdev: 0.04667370101995412 with: {'nodes': 30, 'optimizer': 'SGD'}
Means: 0.7623762488365173, Stdev: 0.024252366463151962 with: {'nodes': 30, 'optimizer': 'Adam'}
Means: 0.5808580915133158, Stdev: 0.1469295693386048 with: {'nodes': 40, 'optimizer': 'SGD'}
Means: 0.7326732675234476, Stdev: 0.04277718116912489 with: {'nodes': 40, 'optimizer': 'Adam'}


In [ ]:
# tuned accuracy is .778

In [330]:
nn_architecture = [
    {"input_dim": 2, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 6, "activation": "relu"},
    {"input_dim": 6, "output_dim": 4, "activation": "relu"},
    {"input_dim": 4, "output_dim": 1, "activation": "sigmoid"},
]

def init_layers(nn_architecture, seed = 99):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
        
    print('init param_values')
    print(params_values)
    return params_values

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0
    return dZ


def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr


def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
    
    return A_curr, memory


def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

# an auxiliary function that converts probability into class
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()


def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)

    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat))
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values


def update(params_values, grads_values, nn_architecture, learning_rate):
    for layer_idx, layer in enumerate(nn_architecture):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values

def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values, cost_history, accuracy_history

In [313]:
X.shape, y.shape
X = X.to_numpy()
y = y.to_numpy()

In [331]:
print(type(nn_architecture))
# Training
params_values = train(np.transpose(X), np.transpose(y.reshape((y.shape[0], 1))), nn_architecture, 10000, 0.01)[0]

<class 'list'>
init param_values
{'W0': array([[-0.04167578, -0.00562668],
       [-0.21361961,  0.16402708],
       [-0.17934356, -0.08417474],
       [ 0.05028814, -0.12452881]]), 'b0': array([[-0.10579522],
       [-0.09090076],
       [ 0.0551454 ],
       [ 0.2292208 ]]), 'W1': array([[ 0.00415394, -0.11179254,  0.05390583, -0.05961597],
       [-0.00191305,  0.11750012, -0.07478709,  0.00090253],
       [-0.08781079, -0.01564342,  0.02565705, -0.0988779 ],
       [-0.0338822 , -0.0236184 , -0.0637655 , -0.11876123],
       [-0.14212172, -0.01534952, -0.0269057 ,  0.22313668],
       [-0.24347676,  0.01127265,  0.03704445,  0.13596339]]), 'b1': array([[ 5.01857207e-02],
       [-8.44213704e-02],
       [ 9.76147160e-07],
       [ 5.42352572e-02],
       [-3.13508197e-02],
       [ 7.71011738e-02]]), 'W2': array([[-0.18680907,  0.17311847,  0.1467678 , -0.03356773,  0.06113408,
         0.00479706],
       [-0.08291353,  0.00877102,  0.10003659, -0.03810925, -0.03756694,
        -0

ValueError: shapes (6,4) and (2,10000) not aligned: 4 (dim 1) != 2 (dim 0)

In [317]:

# Prediction
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, nn_architecture)

NameError: name 'X_test' is not defined